<h1>Introduction</h1>
This analysis looks at different angles of the fuel data for the vehicle spreadsheet provided for the project. It looks at fuel in the scope of transmission and vehicle types and looks at the average price for different products.

These are the different libraries that are necessary to complete the analysis.

In [1300]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
from math import factorial as ft 
import plotly.express as px

This reads the data into a DataFrame and displays the information and the first five rows of the data.

In [1301]:
ice_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/ice-games-practicum/moved_games.csv', encoding = "utf-8")

display(ice_data.head())
ice_data.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


<h2>Data PreProcessing</h2>

This code checks for any fully duplicate rows. There appear to be none.

In [1302]:
print(ice_data.duplicated().sum())

0


I replaced all the column names with the lowercase version of the name.

In [1303]:
ice_data = ice_data.rename(columns={
    'Name': 'name', 
    'Platform': 'platform',
    'Year_of_Release': 'year_of_release',
    'Genre': 'genre',
    'NA_sales': 'na_sales',
    'EU_sales': 'eu_sales',
    'JP_sales': 'jp_sales',
    'Other_sales': 'other_sales',
    'User_Score': 'user_score',
    'Critic_Score': 'critic_score',
    'Rating': 'rating'
})
print(ice_data.columns)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


This code checks for duplicates in the name data by converting all the names to lowercase values and dropping any duplicates that also have a duplicate year of release and platform. This is because some games have new releases in different years and games could be released on different platforms.

In [1304]:
ice_data.info()
print(ice_data['name'].value_counts())
print(ice_data['name'].unique())
ice_data['name_lowercase'] = ice_data['name'].str.lower()
print(ice_data['name'].value_counts())
print(ice_data['name_lowercase'].unique())

ice_data = ice_data.drop_duplicates(subset=['name_lowercase', 'year_of_release', 'platform']).reset_index(drop=True)

ice_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
Need for Speed: Most Wanted                         12
Ratatouille                                          9
LEGO Marvel Super Heroes                             9
FIFA 14                                              9
Madden NFL 07        

This code checks for names that are missing, then grabs the rows where the name is missing to view for analysis.

It looks like quite a bit of information is missing about this row; however, there were in fact sales in Europe and North America for this game, and the platform is valid and used for other games. Therefore, I decided to keep the row and add the string "Unknown Name" to the name field.

In [1305]:
print(ice_data['name'].isna().sum())
print(ice_data.query("name.isna()"))
print(ice_data['platform'].value_counts().head(25))
ice_data['name'] = ice_data['name'].fillna('Unknown Name')
print(ice_data['name'].isna().sum())


1
    name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659  NaN      GEN           1993.0   NaN      1.78      0.53       0.0   

     other_sales  critic_score user_score rating name_lowercase  
659         0.08           NaN        NaN    NaN            NaN  
PS2     2161
DS      2151
PS3     1330
Wii     1320
X360    1262
PSP     1209
PS      1197
PC       974
XB       824
GBA      822
GC       556
3DS      520
PSV      430
PS4      392
N64      319
XOne     247
SNES     239
SAT      173
WiiU     147
2600     133
NES       98
GB        98
DC        52
GEN       28
NG        12
Name: platform, dtype: int64
0


I looked at sample rows that are missing the year of release and I noticed that some have the year in the name field. For those rows, I used the year listed in the name to fill the year of release. For the other rows, I replaced the missing year of release with median year of release. For all rows I converted the datatype to datetime and made sure to isolate just the year.

In [1306]:
print(ice_data['year_of_release'].isna().sum())

#fill year from name
ice_data['year_of_release'] = ice_data['year_of_release'].where(ice_data['name'] != 'PES 2009: Pro Evolution Soccer', 2009)
ice_data['name_year'] = ice_data.query("year_of_release.isna() and (name.str.contains('200') or name.str.contains('19'))")['name'].str[-4:]
ice_data['year_of_release'] = ice_data['year_of_release'].where((ice_data['year_of_release'].notna() & ice_data['name_year'].isna()), ice_data['name_year'])

#check work
print(ice_data['year_of_release'].isna().sum())

#fill missing fields by median
year_median = ice_data['year_of_release'].median()
year_median = round(year_median, 0)
ice_data['year_of_release'] = ice_data['year_of_release'].where((ice_data['year_of_release'].notna()) , year_median) 

#convert datatype
ice_data['year_of_release'] = ice_data['year_of_release'].astype(int)
ice_data['year_of_release'] = pd.to_datetime(ice_data['year_of_release'], format='%Y')
ice_data['year_of_release'] = ice_data['year_of_release'].dt.year

#check work
print(ice_data['year_of_release'].isna().sum())



269
254
0


This code checks for genres that are missing, then grabs the rows where the genre is missing to view for analysis.

It looks like this is the same row that was missing the name. Based on my previous analysis, I decided to keep the row and add the string "Unknown Genre" to the genre field.

In [1307]:
print(ice_data['genre'].isna().sum())
print(ice_data.query("genre.isna()"))
ice_data['genre'] = ice_data['genre'].fillna('Unknown Genre')
print(ice_data['genre'].isna().sum())


1
             name platform  year_of_release genre  na_sales  eu_sales  \
659  Unknown Name      GEN             1993   NaN      1.78      0.53   

     jp_sales  other_sales  critic_score user_score rating name_lowercase  \
659       0.0         0.08           NaN        NaN    NaN            NaN   

    name_year  
659       NaN  
0


I looked at sample rows that are missing the critic score and I did not see any pattern. It is likely these are simply games that did not have a critic score yet. This code calculates the mean critic score by year of release and fills in the missing critic score data with the appropriate mean by year.

In [1308]:
#view data and compare
print(ice_data['critic_score'].isna().sum())
print(ice_data.query("critic_score.isna()").sample(5))
print(ice_data.query("critic_score.isna() and user_score.notna()")['name'].count())

#calculate critic score by year and fill missing values
critic_score_mean = ice_data.groupby('year_of_release')['critic_score'].mean()
critic_score_mean = critic_score_mean.fillna(0)
critic_score_mean = critic_score_mean.reset_index().rename(columns={0: 'year_of_release', 'critic_score': 'mean_critic_score'})
ice_data = ice_data.merge(critic_score_mean, on='year_of_release', how='left')
ice_data['critic_score'] = ice_data['critic_score'].fillna(ice_data['mean_critic_score'])

#check work
print(ice_data['critic_score'].isna().sum())


8577
                                                name platform  \
9172                             The Bombing Islands       PS   
11952             Famicom Mini: Nazo no Murasame-Jou      GBA   
4323   World Soccer Winning Eleven 5 Final Evolution      PS2   
9838             Dodge Racing: Charger vs Challenger      Wii   
14570                                       Terraria     WiiU   

       year_of_release     genre  na_sales  eu_sales  jp_sales  other_sales  \
9172              1997  Platform      0.08      0.05      0.00         0.01   
11952             2004    Action      0.00      0.00      0.07         0.00   
4323              2001    Sports      0.00      0.00      0.45         0.00   
9838              2009    Racing      0.11      0.00      0.00         0.01   
14570             2016    Action      0.00      0.03      0.00         0.00   

       critic_score user_score rating  \
9172            NaN        NaN    NaN   
11952           NaN        NaN    NaN   
4323  

I looked at sample rows that are missing the rating, and I noticed that any game that is missing a rating is also missing a user score, but I am actually not sure if there is a conclusion to be drawn there. The code then fills blank values with "Rating Unknown".

In [1309]:
print(ice_data['rating'].isna().sum())
print(ice_data.query("rating.isna()").sample(5))
print(ice_data.query("rating.isna() and user_score.notna()")['name'].count())
ice_data['rating'] = ice_data['rating'].fillna('Rating Unknown')
print(ice_data['rating'].isna().sum())

6765
                                         name platform  year_of_release  \
1950                                NFL Blitz      N64             1998   
12571  Smile PreCure! Let's Go! Marchen World      3DS             2012   
543                                 Minecraft     XOne             2014   
11278                       Great Party Games      Wii             2010   
10205                          Batman & Robin       PS             1998   

        genre  na_sales  eu_sales  jp_sales  other_sales  critic_score  \
1950   Sports      1.02      0.04      0.00         0.01     81.821429   
12571  Action      0.00      0.00      0.06         0.00     72.953125   
543      Misc      1.61      0.90      0.00         0.25     71.065134   
11278    Misc      0.00      0.07      0.00         0.01     67.482000   
10205  Action      0.06      0.04      0.00         0.01     81.821429   

      user_score rating                          name_lowercase name_year  \
1950         NaN    Na

Two of the rows has a user score of "tbd". I am guessing that this means that a user has not giving this a score or user rating yet. To mitigate this, I averaged all the unique ratings values together and replaced the "tbd" values with this average. The remaining code calculates the mean user score by year of release and fills in the missing (NaN) user score data with the appropriate mean by year.

In [1310]:
print(ice_data['user_score'].isna().sum())
print(ice_data['user_score'].unique())
print(ice_data.query("user_score == 'tbd'"))

#calculate overall user_score mean to replace "tbd" value
user_score_dropna = ice_data['user_score'].dropna()
score_list = list(user_score_dropna.unique())
score_list.remove('tbd')
score_float_list = [float(score) for score in score_list]
score_mean = sum(score_float_list) / len(score_float_list)
ice_data['user_score'] = ice_data['user_score'].where((ice_data['user_score'] != 'tbd') , score_mean) 

#fill NaN values with user_score mean by year
ice_data['user_score'] = ice_data['user_score'].astype(float)
user_score_mean = ice_data.groupby('year_of_release')['user_score'].mean()
user_score_mean = user_score_mean.fillna(0)
user_score_mean = user_score_mean.reset_index().rename(columns={0: 'year_of_release', 'user_score': 'mean_user_score'})
ice_data = ice_data.merge(user_score_mean, on='year_of_release', how='left')
ice_data['user_score'] = ice_data['user_score'].fillna(ice_data['mean_user_score'])

#check work
print(ice_data['user_score'].isna().sum())

6700
['8' nan '8.3' '8.5' '6.6' '8.4' '8.6' '7.7' '6.3' '7.4' '8.2' '9' '7.9'
 '8.1' '8.7' '7.1' '3.4' '5.3' '4.8' '3.2' '8.9' '6.4' '7.8' '7.5' '2.6'
 '7.2' '9.2' '7' '7.3' '4.3' '7.6' '5.7' '5' '9.1' '6.5' 'tbd' '8.8' '6.9'
 '9.4' '6.8' '6.1' '6.7' '5.4' '4' '4.9' '4.5' '9.3' '6.2' '4.2' '6' '3.7'
 '4.1' '5.8' '5.6' '5.5' '4.4' '4.6' '5.9' '3.9' '3.1' '2.9' '5.2' '3.3'
 '4.7' '5.1' '3.5' '2.5' '1.9' '3' '2.7' '2.2' '2' '9.5' '2.1' '3.6' '2.8'
 '1.8' '3.8' '0' '1.6' '9.6' '2.4' '1.7' '1.1' '0.3' '1.5' '0.7' '1.2'
 '2.3' '0.5' '1.3' '0.2' '0.6' '1.4' '0.9' '1' '9.7']
                                           name platform  year_of_release  \
119                               Zumba Fitness      Wii             2010   
301              Namco Museum: 50th Anniversary      PS2             2005   
520                             Zumba Fitness 2      Wii             2011   
645                                uDraw Studio      Wii             2010   
657    Frogger's Adventures: Temple of th

This code creates a total sales column and adds the other three regions together to determine the value

In [1311]:
ice_data['total_sales'] = ice_data['na_sales'] + ice_data['eu_sales'] + ice_data['jp_sales']

This code drops all the extra columns created to clean the data.

In [1312]:
ice_data.drop(['name_lowercase', 'name_year', 'mean_critic_score', 'mean_user_score'], axis=1, inplace=True)



This code checks the info on the dataframe and a random ten rows of the DataFrame

In [1313]:
ice_data.info()
display(ice_data.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16712
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int64  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     16713 non-null  float64
 9   user_score       16713 non-null  float64
 10  rating           16713 non-null  object 
 11  total_sales      16713 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 1.7+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
11109,Kenka Banchou 2: Full Throttle,PS2,2007,Action,0.00,0.00,0.09,0.00,66.314593,6.444315,Rating Unknown,0.09
7960,Warriors: Legends of Troy,PS3,2011,Action,0.06,0.04,0.07,0.02,44.000000,7.000000,M,0.17
6732,Indiana Jones and the Staff of Kings,DS,2009,Action,0.16,0.07,0.00,0.02,50.000000,6.800000,T,0.23
13543,Kamen Rider: Battride War Genesis,PS3,2016,Action,0.00,0.00,0.04,0.00,73.155172,6.441358,Rating Unknown,0.04
236,Forza Motorsport 4,X360,2011,Racing,2.08,1.97,0.06,0.46,91.000000,8.200000,E,4.11
14457,Mahjong 300,DS,2011,Puzzle,0.00,0.03,0.00,0.00,68.692000,6.129688,Rating Unknown,0.03
8568,Rio,DS,2011,Misc,0.12,0.03,0.00,0.01,59.000000,4.989474,E,0.15
11494,Super Robot Taisen OG Saga: Endless Frontier,DS,2008,Role-Playing,0.07,0.00,0.00,0.01,73.000000,9.000000,T,0.07
13096,Dokapon Journey,DS,2008,Role-Playing,0.05,0.00,0.00,0.00,60.000000,7.000000,E10+,0.05
11642,Zoo Tycoon (2013),X360,2013,Simulation,0.02,0.06,0.00,0.01,71.278388,6.401072,Rating Unknown,0.08


<h2>Data Analysis</h2>

Look at how many games were released in different years. Is the data for every period significant?

In [1314]:
games_per_year = ice_data.groupby('year_of_release')['name'].count()

print(games_per_year)

year_of_release
1980       9
1981      46
1982      36
1983      17
1984      14
1985      14
1986      21
1987      16
1988      15
1989      17
1990      16
1991      41
1992      43
1993      61
1994     121
1995     219
1996     263
1997     289
1998     379
1999     338
2000     350
2001     482
2002     830
2003     779
2004     764
2005     941
2006    1008
2007    1452
2008    1426
2009    1430
2010    1255
2011    1136
2012     652
2013     544
2014     581
2015     606
2016     502
Name: name, dtype: int64


It looks the data starts to be significant in 1994, when the count is over 100. It could be that data did not start to be reported consistently before that time.

Look at how sales varied from platform to platform. Choose the platforms with the greatest total sales and build a distribution based on data for each year. Find platforms that used to be popular but now have zero sales. How long does it generally take for new platforms to appear and old ones to fade?

In [1366]:
#create bar chart of all total sales
grp = ice_data.groupby(['platform', 'year_of_release'])
sales_per_platform = grp['total_sales'].sum()
sales_per_platform = sales_per_platform.reset_index().rename(columns={0: 'platform', 1: 'year_of_release', 'total_sales': 'total_sales'})
sales_platform_bar = px.bar(sales_per_platform,
                            x='platform',
                            y='total_sales',
                            color='year_of_release',
                            labels={
                                    'platform': 'Platform',
                                    'total_sales': 'Total Sales (in USD millions)',
                                    'year_of_release': 'Year of Release'},                              
                            title='Total Sales (in USD millions) per Platform per Year')

#grab top platforms and create bar chat of their total sales only
sorted_by_total_sales = sales_per_platform.sort_values(by='total_sales', ascending=False)
top_platforms = sorted_by_total_sales['platform'].head(20).reset_index().rename(columns={0: 'platform'})
unique_platforms = top_platforms['platform'].unique()
unique_platforms_list = list(unique_platforms)
print(unique_platforms_list)

top_sales_platform = sales_per_platform[sales_per_platform['platform'].isin(unique_platforms_list)]
top_sales_platform_bar = px.bar(top_sales_platform,
                                x='platform',
                                y='total_sales',
                                color='year_of_release',
                                labels={
                                        'platform': 'Platform',
                                        'total_sales': 'Total Sales (in USD millions)',
                                        'year_of_release': 'Year of Release'},                              
                                title='Total Sales (in USD millions) for Top Platform per Year')

#display results
sales_platform_bar.show()
top_sales_platform_bar.show()
print(top_sales_platform)

['Wii', 'PS2', 'PS', 'X360', 'DS', 'PS3']


    platform  year_of_release  total_sales
27        DS             1985         0.02
28        DS             2004        15.91
29        DS             2005       119.34
30        DS             2006       111.64
31        DS             2007       138.96
..       ...              ...          ...
226     X360             2012        91.45
227     X360             2013        80.82
228     X360             2014        31.61
229     X360             2015        10.86
230     X360             2016         1.40

[68 rows x 3 columns]


Determine what period you should take data for. To do so, look at your answers to the previous questions. The data should allow you to build a prognosis for 2017.